In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from jquants_derivatives import database, Client, Option, plot_volatility
from skewkurtosis import SkewKurtosis

In [2]:
skew_kurtosis = pd.read_csv("skew_kurtosis0.25.csv", index_col=0)
low_skew = skew_kurtosis.loc[:, "skew"] < 0.02
entry_low_skew = low_skew.shift(1)
entry_low_skew.iloc[0] = False
entry_days = entry_low_skew.loc[entry_low_skew].index

In [3]:
entry_days

Index(['2016-07-22', '2016-08-01', '2016-08-29', '2016-12-07', '2016-12-08',
       '2016-12-13', '2016-12-16', '2016-12-21', '2017-01-27', '2017-01-30',
       ...
       '2023-09-01', '2023-09-04', '2023-09-05', '2023-09-07', '2023-09-08',
       '2023-09-11', '2023-09-12', '2023-09-13', '2023-09-14', '2023-09-15'],
      dtype='object', length=250)

In [4]:
skew_kurtosis = {}
for entry_day in entry_days:
    skew_kurtosis[entry_day] = SkewKurtosis(entry_day)

In [5]:
def get_payoff_call(k, s, price):
    if (s - k) <= 0:
        return -price
    else:
        return (s - k) - price

In [6]:
def get_long_butterfly(skew_kurt_data, target_delta=0.25, strike_width=250):
    date = skew_kurt_data.data.date
    near_maturity = skew_kurt_data.data.contract_month[0]
    contract_df = skew_kurt_data.data.contracts_dfs[near_maturity].set_index(
        "StrikePrice", drop=False
    )
    delta = contract_df.loc[:, "Delta"]
    k = contract_df.loc[:, "StrikePrice"]
    sq_price = skew_kurt_data.data.sq_price[near_maturity]
    if np.isnan(sq_price):
        raise ValueError
    target_k = skew_kurt_data.get_interp_strike(delta, target_delta)
    entry_k = k[abs(k - target_k).idxmin()]
    portfolio_k_price = contract_df.loc[
        [entry_k - strike_width, entry_k, entry_k + strike_width], "TheoreticalPrice"
    ]
    portfolio = (
        pd.DataFrame(portfolio_k_price)
        .assign(qty=[1, -2, 1])
        .assign(date=date)
        .assign(sq=sq_price)
    )
    return portfolio

In [7]:
def get_portfolio(target_delta=0.25, strike_width=250):
    long_butterfly = []
    for entry_day in entry_days:
        try:
            long_butterfly.append(
                get_long_butterfly(skew_kurtosis[entry_day], target_delta, strike_width)
            )
        except (KeyError, ValueError):
            pass
    portfolio = pd.concat(long_butterfly).reset_index().set_index("date")
    return portfolio

In [8]:
portfolio = get_portfolio(target_delta=0.35)
portfolio.head()

,StrikePrice,TheoreticalPrice,qty,sq
date,,,,
2016-07-22,16875.0,335.0008,1,16926.6
2016-07-22,17125.0,249.9999,-2,16926.6
2016-07-22,17375.0,179.9995,1,16926.6
2016-08-01,16625.0,289.9999,1,16926.6
2016-08-01,16875.0,164.9997,-2,16926.6


In [9]:
pl = portfolio.apply(
    lambda x: get_payoff_call(x["StrikePrice"], x["sq"], x["TheoreticalPrice"])
    * x["qty"],
    axis=1,
)
pl.tail()

date
2023-09-04    300.0010
2023-09-04    -74.9995
2023-09-05    -78.5998
2023-09-05    239.9990
2023-09-05    -53.0002
dtype: float64

In [11]:
pl_groupby_day = pl.groupby(level=0).sum()
pl_groupby_day.tail()

date
2023-08-29    -29.9998
2023-08-30    -23.0003
2023-09-01    261.6000
2023-09-04     91.4010
2023-09-05    108.3990
dtype: float64

In [12]:
px.violin(pl_groupby_day)

In [13]:
px.line(pl_groupby_day.cumsum())

In [14]:
portfolio.to_csv("long_butterfly_portfolio.csv")